In [ ]:
import os
import numpy as np
import pandas as pd
import requests
from PIL import Image
from sklearn.cluster import KMeans
from IPython.display import display, HTML

# CSV 파일 로드
csv_file_path = '/content/a9782c58a4373a92.csv'
df = pd.read_csv(csv_file_path)
df=df.tail()

# 데이터프레임의 컬럼 출력 (확인용)
print("데이터프레임의 컬럼:", df.columns)

# '선택한 색상' 열을 빈 값으로 초기화
df['선택한 색상'] = None

# 임시 디렉토리 생성
if not os.path.exists('temp_images'):
    os.makedirs('temp_images')

# 중앙 부분의 색상을 추출하는 함수 정의
def get_center_colors(image_url, center_fraction=0.2, n_colors=4):
    try:
        print(f"URL 처리 중: {image_url}")  # URL 로그 출력
        response = requests.get(image_url, timeout=10, verify=False)  # SSL 인증서 검증을 건너뜁니다.
        if response.status_code == 200:
            # 이미지 파일을 디스크에 저장
            img_path = os.path.join('temp_images', os.path.basename(image_url))
            print(f"이미지 저장 중: {img_path}")
            with open(img_path, 'wb') as f:
                f.write(response.content)

            # 저장된 이미지 파일이 있는지 확인
            if os.path.exists(img_path):
                print(f"이미지가 성공적으로 저장됨: {img_path}")
                # 저장된 이미지 파일 열기
                try:
                    img = Image.open(img_path)
                    img = img.convert('RGB')
                    width, height = img.size
                    left = int((1 - center_fraction) * width / 2)
                    top = int((1 - center_fraction) * height / 2)
                    right = int((1 + center_fraction) * width / 2)
                    bottom = int((1 + center_fraction) * height / 2)
                    img_cropped = img.crop((left, top, right, bottom))

                    img_array = np.array(img_cropped)
                    img_array = img_array.reshape((img_array.shape[0] * img_array.shape[1], 3))

                    kmeans = KMeans(n_clusters=n_colors)
                    kmeans.fit(img_array)
                    colors = kmeans.cluster_centers_
                    rgb_colors = [list(map(int, color)) for color in colors]

                    return rgb_colors
                except Exception as e:
                    print(f"이미지 열기 또는 처리 중 오류: {img_path}, 오류: {e}")
                    return None
            else:
                print(f"이미지 파일을 찾을 수 없음: {img_path}")
                return None
        else:
            print(f"URL에서 이미지를 가져오는 데 실패: {image_url}")
            return None
    except Exception as e:
        print(f"이미지 처리 중 오류: {image_url}, 오류: {e}")
        return None

# 주요 색상 추출 및 데이터프레임에 추가
df['중앙 색상들'] = df['이미지'].apply(lambda x: get_center_colors(x, n_colors=4))

# 사용자 인터페이스를 통해 색상 선택
selected_colors = []

def display_image_and_colors(image_url, colors):
    display(HTML(f'<img src="{image_url}" alt="Image" style="max-width:200px;"/>'))
    color_boxes = ''.join([f'<div style="width:50px; height:50px; display:inline-block; background-color:rgb({color[0]},{color[1]},{color[2]});"></div>' for color in colors])
    display(HTML(color_boxes))

for index, row in df.iterrows():
    img_url = row['이미지']
    colors = row['중앙 색상들']

    # 이미지 및 색상 박스 표시
    display_image_and_colors(img_url, colors)

    # 색상 선택 메시지
    print("다음 색상 중 하나를 선택하세요 (선택지에 없으면 0 입력):")
    for i, color in enumerate(colors):
        print(f"{i + 1}: 색상")

    # 사용자 입력 받기
    selected_index = int(input("선택한 색상의 번호를 입력하세요: ")) - 1

    if selected_index == -1:
        selected_color = None
    else:
        selected_color = colors[selected_index]

    # 선택된 색상을 데이터프레임에 저장
    df.at[index, '선택한 색상'] = selected_color

# 선택된 색상을 포함한 데이터프레임 저장
output_csv_path = 'flower_data_with_selected_colors.csv'
df.to_csv(output_csv_path, index=False)
print(f"CSV 파일이 {output_csv_path}로 저장되었습니다.")

# 임시 디렉토리 삭제
import shutil
if (os.path.exists('temp_images')):
    shutil.rmtree('temp_images')